<img src="./img/metro_madrid.jpg" alt="drawing" width="500"/>

# Transporte y renta en la Comunidad de Madrid

### TEMA

Este proyecto busca arrojar luz sobre los patrones de comportamiento en el transporte de los ciudadanos de la Comunidad de Madrid, estudiando posibles vínculos con su renta. Para ello, se utiliza la [Encuesta de Movilidad del Consorcio Regional](https://datos.comunidad.madrid/dataset/resultados-edm2018) de Transportes de Madrid (CRTM), realizada en el 2018.

Para los cálculos de la renta, se utilizan diferentes tablas extraídas de la [Encuesta de Condiciones de Vida](https://ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176807&menu=ultiDatos&idp=1254735976608) del INE, para el año 2018

### HIPÓTESIS

La hipótesis principal de este proyecto es que **existe una relación entre la renta y el uso del transporte en la Comunidad de Madrid**. En concreto, se buscará contrastar:
+ Si las rentas más altas eligen el coche como primera opción para desplazamientos laborales y por placer.
+ Las rentas bajas, para trabajar, utilizan más el transporte público que el coche.
+ Las mujeres utilizan más el transporte público para desplazamientos laborales, sin tener en cuenta la renta.
    + Lo que, unido al segundo punto, explicaría que las mujeres no usan más el transporte público por deseo sino por necesidad.
+ Los hombres utilizan más el transporte privado para desplazamientos por placer


### SUPUESTOS
Dados los datos utilizados, hemos de tener en cuenta varios puntos:
+ Se está utilizando la última encuesta publicada por el CRTM (en serio).
    + Sí se cuentan con los datos de uso del Metro de Madrid, que en 2023 volvieron a niveles pre-pandemia. Por tanto, podemos suponer que no han cambiado las pautas de comportamiento en un nivel agregado
+ No contamos con la renta de los encuestados. Para ello generaremos un dato *proxy*, que estimaremos en función de las condiciones socioeconómicas de los encuestados (edad, ocupación, género, grado de educación, y si cuenta con coche).
    + Renta != Riqueza. Estimaremos la cantidad de **generar** , no de **poseer**. Una persona sin trabajo pero con patrimonio no quedará reflejada en este estudio.
    + Por disponibilidad de los datos, se utilizarán las medias de las rentas. En función de disponibilidad de los datos, es posible que se recoja la media española en lugar de la de la Comunidad de Madrid.
+ Los datos sólo tienen en cuenta días estrictamente laborales (Lunes-Jueves)
+ Si bien no tiene impacto a primera vista sobre las hipótesis, se realizará un estudio de impacto de la **meteorología** sobre el uso de unos u otros medios transportes.


## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

In [51]:
import hashlib
import requests
import datetime
import pandas as pd
### Importar openpyxl
# guarda en variables los datasets y su fuente
df_transpk = pd.DataFrame(pd.read_csv("./data/raw/kaggle_public_transp.csv"))
fuente_1a = "https://www.kaggle.com/datasets/dataguapa/madrid-public-transportation-data-2018"

df_transpc = pd.DataFrame(pd.read_excel("./data/raw/EDM2018INDIVIDUOS.xlsx", sheet_name = 'INDIVIDUOS'))
fuente_1 = "https://www.kaggle.com/datasets/dataguapa/madrid-public-transportation-data-2018"

df_hogares = pd.DataFrame(pd.read_csv("./data/raw/ine_gasto_hogares.csv", sep=";"))
fuente_2 = "https://www.ine.es/jaxiT3/Tabla.htm?t=24900"

df_ingresos = pd.DataFrame(pd.read_csv("./data/raw/ine_madrid_fuentes_ingreso.csv",sep=";"))
fuente_3 = "https://ine.es/jaxiT3/Tabla.htm?t=53687"

df_renta_es = pd.DataFrame(pd.read_csv("./data/raw/ine_renta_edad_sexo.csv",encoding='latin1',sep=";"))
fuente_4 = "https://ine.es/jaxiT3/Tabla.htm?t=9942"

df_educacion = pd.DataFrame(pd.read_excel("./data/raw/ine_renta_educacion.xlsx", sheet_name="educación",skiprows=2,index_col=0,nrows=5)) ### Terminar de apañar
fuente_5 = "https://ine.es/ss/Satellite?c=INESeccion_C&cid=1259944504067&p=1254735110672&pagename=ProductosYServicios%2FPYSLayout&param1=PYSDetalleFichaIndicador&param3=1259937499084"

In [54]:
df_transpc['id']=df_transpc['ID_HOGAR'] + df_transpc['ID_IND']
df_transpc

,ID_HOGAR,ID_IND,C2SEXO,EDAD_FIN,ELE_G_POND,C4NAC,C5CAM,C6CARNE,C7ESTUD,C8ACTIV,...,DDIA,DMES,DANNO,DIASEM,DNOVIAJO,C11ZT1259,C12ZT1259,CPMR,TIPO_ENCUESTA,id
0,189,1,1,28,66.304668,1,1,4,4,1,...,25,4,2018,3,NaN,104-001B,NaN,2,CAPI,190
1,189,2,2,23,73.500000,1,1,4,4,4,...,25,4,2018,3,2.0,NaN,NaN,2,CAPI,191
2,244,1,1,36,66.304668,2,1,4,4,1,...,26,4,2018,4,NaN,104-001B,NaN,2,CAPI,245
3,244,2,2,35,69.473571,2,1,1,3,4,...,26,4,2018,4,NaN,NaN,NaN,2,CAPI,246
4,324,1,1,81,79.174900,1,1,4,2,3,...,25,4,2018,3,NaN,NaN,NaN,2,CAPI,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,6011027,1,2,55,49.726989,1,1,4,7,1,...,4,6,2018,1,NaN,079-16-431,NaN,2,CATI,6011028
85060,6011053,1,1,49,66.566789,1,1,5,7,1,...,4,6,2018,1,NaN,079-16-431,NaN,2,CATI,6011054
85061,6011053,2,1,6,64.450350,1,1,1,1,6,...,4,6,2018,1,NaN,NaN,079-07-121,2,CATI,6011055
85062,6011059,1,1,14,64.450350,1,1,1,2,6,...,4,6,2018,1,NaN,NaN,079-16-437,2,CATI,6011060


Muestra mediante un head() los principales datasets con los que vas a trabajar

In [45]:
###DF Principal
df_transpk.head()

,mode_main,distance,main_reason,week_day,age,education,female,main_activity,license,cars,temp,precip,wind
0,11,6.660580,2,3,28,1,0,1,1,1,20.8,0.0,2.2
1,11,6.586500,2,4,36,1,0,1,1,1,20.2,0.0,2.5
2,11,0.408073,10,4,72,1,0,3,1,1,20.2,0.0,2.5
3,11,0.695542,2,4,30,1,0,1,1,1,20.2,0.0,2.5
4,11,5.087340,8,4,30,1,0,1,1,1,20.2,0.0,2.5


In [46]:
df_transpk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76759 entries, 0 to 76758
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mode_main      76759 non-null  int64  
 1   distance       76759 non-null  float64
 2   main_reason    76759 non-null  int64  
 3   week_day       76759 non-null  int64  
 4   age            76759 non-null  int64  
 5   education      76759 non-null  int64  
 6   female         76759 non-null  int64  
 7   main_activity  76759 non-null  int64  
 8   license        76759 non-null  int64  
 9   cars           76759 non-null  int64  
 10  temp           76759 non-null  float64
 11  precip         76759 non-null  float64
 12  wind           62556 non-null  float64
dtypes: float64(4), int64(9)
memory usage: 7.6 MB


In [12]:
#Df auxiliar 1
df_hogares.head(5)

,Gastos medios y distribución porcentual,Quintil de gasto,Códigos de gasto (2 dígitos),Periodo,Total
0,Gasto medio por hogar,Total,Índice general,2023,"32.616,66"
1,Gasto medio por hogar,Total,Índice general,2022,"31.567,71"
2,Gasto medio por hogar,Total,Índice general,2021,"29.243,61"
3,Gasto medio por hogar,Total,Índice general,2020,"26.995,76"
4,Gasto medio por hogar,Total,Índice general,2019,"30.242,76"


In [13]:
df_ingresos.head()


,Comunidades y Ciudades Autonomas,Provincias,Islas,Distribuci�n por fuente de ingresos,Periodo,Total
0,"Madrid, Comunidad de",NaN,NaN,Renta bruta media por persona,2018,"19.042,0"
1,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: salario,2018,"12.368,0"
2,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: pensiones,2018,"3.208,0"
3,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: prestaciones por desempleo,2018,"228,0"
4,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: otras prestaciones,2018,"534,0"


In [14]:
df_renta_es.head()

,Sexo,Edad,Renta anual neta media por persona y por unidad de consumo,Periodo,Total
0,Ambos sexos,Menores de 16 años,Renta neta media por persona,2018,8.919
1,Ambos sexos,De 16 a 29 años,Renta neta media por persona,2018,10.156
2,Ambos sexos,De 30 a 44 años,Renta neta media por persona,2018,11.397
3,Ambos sexos,De 45 a 64 años,Renta neta media por persona,2018,12.550
4,Ambos sexos,65 y más años,Renta neta media por persona,2018,12.758


In [16]:
df_educacion.head()

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total,16587.0,17422.0,17327.0,16604.0,16428.0,16023.0,15752.0,15716.0,16129.0,16677.0,17154.0,17570.0,18538.0,18480.0,19562.0
"Nivel 0-2: preescolar, primaria y secundaria de 1ª etapa",13410.0,14071.0,13918.0,13290.0,13231.0,12926.0,12610.0,12435.0,12647.0,13052.0,13373.0,13890.0,14682.0,14357.0,15289.0
Nivel 3-4: secundaria de 2ª etapa y postsecundaria no superior,17342.0,17725.0,17627.0,16978.0,16446.0,16131.0,15713.0,15555.0,16112.0,16399.0,16638.0,16999.0,17799.0,17798.0,18617.0
Nivel 5-8: primer y segundo ciclo de educación superior y doctorado,22946.0,24515.0,24325.0,23285.0,22836.0,21838.0,21372.0,21579.0,22116.0,22783.0,23314.0,23436.0,23924.0,23890.0,25089.0


In [114]:
### Meteorología de AEMET.
df_weather=pd.DataFrame(pd.read_csv('./data/treated/aemet_weather.csv'))
### Cambiamos valores de fecha para que queden acordes a CRTM 
df_weather['fechamerge'] = df_weather['fecha'].str.split("-")
df_weather['fechamerge']
### Expandimos a tres columnas nuevas
df_weather [['year','month','day']] = pd.DataFrame(df_weather.fechamerge.tolist(), index = df_weather.index)
### Quitamos 0s en month y day
df_weather['month'] = df_weather['month'].str.replace("0","")
df_weather['day'] = df_weather['day'].str[0].replace("0","")+df_weather['day'].str[1]
df_weather['day'].head(50)
### Juntamos en año, mes, día
df_weather['fechamerge'] = df_weather['year']+"-"+df_weather['month']+"-"+df_weather['day']
###Comprobamos
df_weather

,Unnamed: 0,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,...,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,fechamerge,year,month,day
0,0,2018-02-01,3195,"MADRID, RETIRO",MADRID,667,"6,6","0,1","2,3",07:40,...,17,69.0,81.0,Varias,47.0,16:00,2018-2-1,2018,2,1
1,1,2018-02-02,3195,"MADRID, RETIRO",MADRID,667,"4,4","0,0","1,2",23:59,...,05,45.0,60.0,Varias,35.0,13:00,2018-2-2,2018,2,2
2,2,2018-02-03,3195,"MADRID, RETIRO",MADRID,667,"5,2","2,3","0,2",02:30,...,24,51.0,76.0,23:50,40.0,13:00,2018-2-3,2018,2,3
3,3,2018-02-04,3195,"MADRID, RETIRO",MADRID,667,"3,6","17,3","1,5",13:00,...,24,90.0,97.0,Varias,76.0,00:00,2018-2-4,2018,2,4
4,4,2018-02-05,3195,"MADRID, RETIRO",MADRID,667,"1,8","14,1","0,8",11:40,...,14,88.0,96.0,Varias,75.0,06:10,2018-2-5,2018,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,2018-06-26,3195,"MADRID, RETIRO",MADRID,667,"29,0","0,0","23,0",06:00,...,17,45.0,62.0,07:00,33.0,15:00,2018-6-26,2018,6,26
146,146,2018-06-27,3195,"MADRID, RETIRO",MADRID,667,"26,9","0,0","21,9",05:00,...,19,43.0,59.0,Varias,31.0,18:20,2018-6-27,2018,6,27
147,147,2018-06-28,3195,"MADRID, RETIRO",MADRID,667,"26,0","0,1","20,9",06:20,...,19,42.0,70.0,Varias,27.0,Varias,2018-6-28,2018,6,28
148,148,2018-06-29,3195,"MADRID, RETIRO",MADRID,667,"26,0","0,0","20,5",05:40,...,Varias,47.0,68.0,23:59,32.0,Varias,2018-6-29,2018,6,29


In [37]:
###Conexión con AEMET

url = 'https://opendata.aemet.es/opendata/sh/b3aa9d28'

res = requests.get(url)
print(res.status_code)
# weather = res.json()
weather_meta = res.json()
weather_meta

200


{'unidad_generadora': 'Servicio del Banco Nacional de Datos Climatológicos',
 'periodicidad': '1 vez al día, con un retardo de 4 días',
 'descripcion': 'Climatologías diarias',
 'formato': 'application/json',
 'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
 'notaLegal': 'https://www.aemet.es/es/nota_legal',
 'campos': [{'id': 'fecha',
   'descripcion': 'fecha del dia (AAAA-MM-DD)',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'indicativo',
   'descripcion': 'indicativo climatológico',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'nombre',
   'descripcion': 'nombre (ubicación) de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'provincia',
   'descripcion': 'provincia de la estación',
   'tipo_datos': 'string',
   'requerido': True},
  {'id': 'altitud',
   'descripcion': 'altitud de la estación en m sobre el nivel del mar',
   'tipo_datos': 'float',
   'unidad': 

In [39]:
### Conversor Aemet a CSV
df_weather_meta = pd.DataFrame(weather_meta)
df_weather_meta.to_csv('./data/aemet_weather_meta.csv')
df_weather_meta

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'fecha', 'descripcion': 'fecha del dia ..."
1,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
2,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'nombre (ubica..."
3,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
4,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."
5,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'tmed', 'descripcion': 'Temperatura med..."
6,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'prec', 'descripcion': 'Precipitación d..."
7,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'tmin', 'descripcion': 'Temperatura Mín..."
8,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'horatmin', 'descripcion': 'Hora y minu..."
9,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'tmax', 'descripcion': 'Temperatura Máx..."


In [115]:
### Encuestas - Conversor fecha para merge
df_transpc[]

,ID_HOGAR,ID_IND,C2SEXO,EDAD_FIN,ELE_G_POND,C4NAC,C5CAM,C6CARNE,C7ESTUD,C8ACTIV,...,DDIA,DMES,DANNO,DIASEM,DNOVIAJO,C11ZT1259,C12ZT1259,CPMR,TIPO_ENCUESTA,id
0,189,1,1,28,66.304668,1,1,4,4,1,...,25,4,2018,3,NaN,104-001B,NaN,2,CAPI,190
1,189,2,2,23,73.500000,1,1,4,4,4,...,25,4,2018,3,2.0,NaN,NaN,2,CAPI,191
2,244,1,1,36,66.304668,2,1,4,4,1,...,26,4,2018,4,NaN,104-001B,NaN,2,CAPI,245
3,244,2,2,35,69.473571,2,1,1,3,4,...,26,4,2018,4,NaN,NaN,NaN,2,CAPI,246
4,324,1,1,81,79.174900,1,1,4,2,3,...,25,4,2018,3,NaN,NaN,NaN,2,CAPI,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,6011027,1,2,55,49.726989,1,1,4,7,1,...,4,6,2018,1,NaN,079-16-431,NaN,2,CATI,6011028
85060,6011053,1,1,49,66.566789,1,1,5,7,1,...,4,6,2018,1,NaN,079-16-431,NaN,2,CATI,6011054
85061,6011053,2,1,6,64.450350,1,1,1,1,6,...,4,6,2018,1,NaN,NaN,079-07-121,2,CATI,6011055
85062,6011059,1,1,14,64.450350,1,1,1,2,6,...,4,6,2018,1,NaN,NaN,079-16-437,2,CATI,6011060
